Устанавливаем библиотеку python-telegram-bot. Ссылка на документацию https://github.com/python-telegram-bot/python-telegram-bot

In [19]:
!pip install python-telegram-bot --upgrade

Requirement already up-to-date: python-telegram-bot in /usr/local/lib/python3.6/dist-packages (13.1)
Requirement already up-to-date: python-telegram-bot in /usr/local/lib/python3.6/dist-packages (13.1)


In [20]:
!pip install pymorphy2



In [21]:
!pip install stop_words

In [22]:
!pip install annoy

In [23]:
!pip install dialogflow

In [40]:
import os
import logging
from telegram import Update
from telegram.ext import Updater, CommandHandler, MessageHandler, Filters, CallbackContext
from pymorphy2 import MorphAnalyzer
from tqdm.notebook import tqdm_notebook
import dialogflow
from stop_words import get_stop_words
from google.colab import drive
from gensim.models import Word2Vec, FastText
import annoy
import pickle
import numpy as np
import string
import regex as re
import random


In [41]:
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [42]:
#!ls '/content/drive/MyDrive/Colab Notebooks/data'

In [43]:
#DATA_PATH = "data/Otvety.txt"
#ANSWERS_PATH = 'data/prepared_answers.txt'
FASTTEXT_MODEL_PATH = '/content/drive/MyDrive/Colab Notebooks/data/ft_model.h5'
RAW_SENTENCES_PATH = '/content/drive/MyDrive/Colab Notebooks/data/sentences.pkl'
INDEX_MAP_PATH = '/content/drive/MyDrive/Colab Notebooks/data/map_ind.pkl'
FT_IND_PATH = '/content/drive/MyDrive/Colab Notebooks/data/ft_ind'
FT_VECTOR_MAX_LEN = 100



In [44]:
morpher = MorphAnalyzer()
stopw = set(get_stop_words("ru"))
exclude = set(string.punctuation)

## Подготовка и обучение FastText модели на ответах mail.ru

In [45]:
model_FT = FastText.load(FASTTEXT_MODEL_PATH)

'''
# Создание индексов FastText
ft_ind = annoy.AnnoyIndex(FT_VECTOR_MAX_LEN ,'angular')

index_map = {}
counter = 0

with open(ANSWERS_PATH, "r", encoding='utf-8') as fout:
    for line in tqdm_notebook(fin):
        num_ft = 0
        spls = line.split("\t")
        index_map[counter] = spls[1]
        question = preprocess_txt(spls[0])
        vector_ft = np.zeros(FT_VECTOR_MAX_LEN)
        for word in question:
            if word in modelFT:
                vector_ft += modelFT[word]
                num_ft += 1
        if num_ft > 0:
            vector_ft = vector_ft / num_ft
        ft_ind.add_item(counter, vector_ft)
            
        counter += 1

# Сохранение просчитанной карты индексов
with open(INDEX_MAP_PATH, 'wb') as fout:
    pickle.dump(index_map, fout)
    
# Построение индексов на 20 деревьях 
ft_ind.build(20)
ft_ind.save(FT_INDEX_PATH)
'''

'\n# Создание индексов FastText\nft_ind = annoy.AnnoyIndex(FT_VECTOR_MAX_LEN ,\'angular\')\n\nindex_map = {}\ncounter = 0\n\nwith open(ANSWERS_PATH, "r", encoding=\'utf-8\') as fout:\n    for line in tqdm_notebook(fin):\n        num_ft = 0\n        spls = line.split("\t")\n        index_map[counter] = spls[1]\n        question = preprocess_txt(spls[0])\n        vector_ft = np.zeros(FT_VECTOR_MAX_LEN)\n        for word in question:\n            if word in modelFT:\n                vector_ft += modelFT[word]\n                num_ft += 1\n        if num_ft > 0:\n            vector_ft = vector_ft / num_ft\n        ft_ind.add_item(counter, vector_ft)\n            \n        counter += 1\n\n# Сохранение просчитанной карты индексов\nwith open(INDEX_MAP_PATH, \'wb\') as fout:\n    pickle.dump(index_map, fout)\n    \n# Построение индексов на 20 деревьях \nft_ind.build(20)\nft_ind.save(FT_INDEX_PATH)\n'

In [46]:
ft_ind = annoy.AnnoyIndex(FT_VECTOR_MAX_LEN, 'angular')
ft_ind.load(FT_IND_PATH)

True

In [47]:
with open(INDEX_MAP_PATH, 'rb') as fout:
    index_map = pickle.load(fout)

In [48]:
def preprocess_txt(line):
    """Препроцессинг строки"""
    spls = "".join(i for i in line.strip() if i not in exclude).split()
    spls = [morpher.parse(i.lower())[0].normal_form for i in spls]
    spls = [i for i in spls if i not in stopw and i != ""]
    return spls

def get_response(question, index, model, index_map):
    
    """Получение ответа из модели fast text"""
    
    question = preprocess_txt(question)
    vect_ft = np.zeros(FT_VECTOR_MAX_LEN)
    num = 0
    for word in question:
        if word in model:
            vect_ft += model[word]
            num += 1
    if num > 0:
        vect_ft = vect_ft / num
    answers = index.get_nns_by_vector(vect_ft, 5) 
    return [index_map[i] for i in answers if len(index_map[i]) < 300] # 


In [49]:
# проверка чата на "ответах mail.ru"
input_text = "Куда мы попали?" 
r = get_response(input_text, ft_ind, model_FT, index_map)
r

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  app.launch_new_instance()
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


['Привет!Чего случилась-заплутала или наших лиц не узнаёшь?!Туда туда родная-всегда радуемся твоему появлению!!. \n',
 'Дааааа! ! Скоростя у них космические!?. \n',
 'Это кое как.. а куда попало, это по обстоятельствам. \n',
 'В клоаку, тобишь в *опу.. \n',
 'Здраствуйте уважаемое "оно" )) <br>вы попали на планету зеиля (дурдом с нулевого года ) <br>))))). \n']

In [50]:
def clean(text):
    
    """Очистка текста от тегов, лишних знаков препинаний, смайлов и т.д."""
    cl_text = random.choice(text)  # Для выбора одного из многочисленных вариантов
    cl_text = re.sub(r'<[^>]*>','', cl_text)
    cl_text = re.sub(r'[\xa0]',' ', cl_text)
    cl_text = re.sub(r'.{1,2} [\n]','', cl_text)
    cl_text = re.sub(r'[\n]','', cl_text)
    cl_text = re.sub(r'\s[.]\s','', cl_text)
    cl_text = re.sub(r"\)\)\)\)", "))", cl_text)
    cl_text = re.sub(r"\)\)", ")", cl_text)
    cl_text = re.sub(r"\) \)", "))", cl_text)
    cl_text = re.sub(r',{2,}','', cl_text)
    return cl_text

Настраиваем бота

Done! Congratulations on your new bot. You will find it at t.me/Evm_gb_bot. You can now add a description, about section and profile picture for your bot, see /help for a list of commands. By the way, when you've finished creating your cool bot, ping our Bot Support if you want a better username for it. Just make sure the bot is fully operational before you do this.

Use this token to access the HTTP API:
1486721342:AAFR7RPnqZHHZSjmRsBmQqaM2Kb5t8bJaes
Keep your token secure and store it safely, it can be used by anyone to control your bot.

For a description of the Bot API, see this page: https://core.telegram.org/bots/api

In [51]:
updater = Updater("1486721342:AAFR7RPnqZHHZSjmRsBmQqaM2Kb5t8bJaes")  # Токен API к Telegram , use_context=True
dispatcher = updater.dispatcher
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = '/content/drive/MyDrive/Colab Notebooks/data/newbot-cfin-91c1d9900dfd.json'# скачнный JSON


DIALOGFLOW_PROJECT_ID = 'newbot-cfin' #PROJECT ID из DialogFlow 
DIALOGFLOW_LANGUAGE_CODE = 'ru' # язык
SESSION_ID = 'Evm_gb_bot'  # ID бота из телеграма
start_msg = 'Новейший и  вероятно не самый сумасшедший бот, готов ответить на большинство вопросов по-русски.'


Переписываем функцию textMessage

In [52]:
def startCommand(update: Update, context: CallbackContext):
    update.message.reply_text('Добрый день!')

    
def textMessage(update: Update, context: CallbackContext):
    
    '''
    # Подлючение бота из dialogflow

    session_client = dialogflow.SessionsClient()
    session = session_client.session_path(DIALOGFLOW_PROJECT_ID, SESSION_ID)
    text_input = dialogflow.types.TextInput(text=update.message.text , language_code=DIALOGFLOW_LANGUAGE_CODE)
    query_input = dialogflow.types.QueryInput(text=text_input)
    
    try:
        response = session_client.detect_intent(session=session, query_input=query_input)
    except InvalidArgument:
         raise 

    text = response.query_result.fulfillment_text 
    if text:
        update.message.reply_text(response.query_result.fulfillment_text)
    else:
        update.message.reply_text('Попробуйте задать вопрос иначе.')
        '''
    input_txt = update.message.text
    print(input_txt)
    response = get_response(input_txt, ft_ind, model_FT, index_map)
    response = clean(response)
    update.message.reply_text(response)
    print(f"Response is: {response}")

In [ ]:
# on different commands - answer in Telegram
'''
dispatcher.add_handler(CommandHandler("start", startCommand1))
dispatcher.add_handler(MessageHandler(Filters.text & ~Filters.command, textMessage1))

# Start the Bot
updater.start_polling()
updater.idle()'''

start_command_handler = CommandHandler('start', startCommand)
text_message_handler = MessageHandler(Filters.text, textMessage)
# Добавляем хендлеры в диспатчер
dispatcher.add_handler(start_command_handler)
dispatcher.add_handler(text_message_handler)
# Начинаем поиск обновлений
updater.start_polling(clean=True)
# Останавливаем бота на Ctrl + C
updater.idle()

Пробуем


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  app.launch_new_instance()
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


Response is: Я бы с удовольствием - большая любительница пирогов!! ! Угостишь??
Конечно. Как насчет малинового, с настойкой?
Response is: Смекта )
Блестяще). Угадай кого топил Герасим?
Response is: так жизнь проходит судьба уносит кого-то вспомнят кого-то не


No error handlers are registered, logging exception.
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/telegram/ext/dispatcher.py", line 432, in process_update
    handler.handle_update(update, self, check, context)
  File "/usr/local/lib/python3.6/dist-packages/telegram/ext/handler.py", line 156, in handle_update
    return self.callback(update, context)
  File "<ipython-input-52-2797657d588b>", line 28, in textMessage
    response = get_response(input_txt, ft_ind, model_FT, index_map)
  File "<ipython-input-48-41903e3a9ab4>", line 16, in get_response
    if word in model:
  File "/usr/local/lib/python3.6/dist-packages/gensim/utils.py", line 1422, in new_func1
    return func(*args, **kwargs)
  File "/usr/local/lib/python3.6/dist-packages/gensim/models/fasttext.py", line 710, in __contains__
    return self.wv.__contains__(word)
  File "/usr/local/lib/python3.6/dist-packages/gensim/models/keyedvectors.py", line 1930, in __contains__
    return any(_ft_h

Как то уклончиво. Сколько км в парсеке?
Ладно, попроще. Сколько весит слон?
Response is: подоидет полно цветов под нее... но самое главное модель верха.... даже классно будет иаркосинии цвет...
Твой любимый рецепт
Response is: салат в лаваш
Прям народный
Response is: Дуримар а зачем тебе, подлечиться пиявками хочешь
